<a href="https://colab.research.google.com/github/radanim/TIL/blob/master/depth_classification_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from glob import glob
from tqdm.auto import tqdm
import re
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

In [ ]:
train_paths = glob('/content/drive/MyDrive/dataset/train/SEM/*/*/*.png')

In [ ]:
# image to np.array
images_train = []

for image_path in tqdm(train_paths): 
    src = cv2.imread(image_path, 0)
#     dst = cv2.GaussianBlur(src, (0, 0), 0.5)
#     images_train.append(dst)
    images_train.append(src)

  0%|          | 0/8398 [00:00<?, ?it/s]

In [ ]:
images_train = np.array(images_train)
images_train.shape

(8398, 72, 48)

In [ ]:
# save
with open('image_train.npy', 'wb') as f:
    np.save(f, images_train)

In [ ]:
# load
with open('image_train.npy', 'rb') as f:
    images_t = np.load(f)

In [ ]:
images_t = np.array(images_t/212, dtype='float32')
images_t = np.expand_dims(images_t, axis=3)
images_t.shape

(8398, 72, 48, 1)

In [ ]:
strat = np.array([re.findall(r'.*(Depth_[0-9]{3}\\site_[0-9]{5}).*', path)[0] for path in train_paths])

IndexError: ignored

In [ ]:
target = np.array([int(re.findall(r'.*Depth_1([1-4])0.*', path)[0])-1 for path in train_paths])

In [ ]:
n_images = len(target)
seed_=2022

In [ ]:
tf.random.set_seed(seed_)

# shuffle
random.seed(seed_)
idx = random.sample(range(n_images), n_images)

images_t_ = images_t[idx,:,:]
target_ = target[idx]
strat_ = strat[idx]

X_train, X_valid, y_train, y_valid = train_test_split(images_t_, target_, test_size = 0.2, stratify=strat_, random_state=2022)

print(f'X_train shape : {X_train.shape}, y_train shape : {y_train.shape}')
print(f'X_valid shape : {X_valid.shape}, y_valid shape : {y_valid.shape}')

In [ ]:
# batch
BATCH_SIZE = 64
AUTOTUNE = tf.data.AUTOTUNE

# train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train, w_train))
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))

train_dataset = train_dataset.batch(BATCH_SIZE, num_parallel_calls=AUTOTUNE).cache().prefetch(buffer_size=AUTOTUNE)
valid_dataset = valid_dataset.batch(BATCH_SIZE, num_parallel_calls=AUTOTUNE).cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
model = Sequential(
    [
        Input(shape=(72, 48, 1)),
                
        Conv2D(32, kernel_size=(3, 3), padding='same', activation="relu"),
        Conv2D(32, kernel_size=(3, 3), padding='same', activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        
        Conv2D(64, kernel_size=(3, 3), padding='same', activation="relu"),
        Conv2D(64, kernel_size=(3, 3), padding='valid', activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.3),
        
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(4, activation="softmax"),
    ]
)

model.summary()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                                 patience=2, min_lr=1e-04)

In [ ]:
model.compile(
    optimizer=Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [ ]:
hist  = model.fit(train_dataset,
                  validation_data=valid_dataset,    
                  verbose =1,
                  epochs=60, 
                  callbacks=[early_stopping, reduce_lr])